# 🧪 Analyse Exploratoire des Données (EDA)

## 📌 Introduction

Dans ce notebook, nous allons réaliser une **analyse exploratoire des données (EDA)** sur un jeu de données. L'objectif est de mieux comprendre la structure des données, les relations entre les variables, les valeurs manquantes, les valeurs aberrantes et d'autres caractéristiques qui nous permettront de préparer les données pour un modèle prédictif.

## 📚 Table des Matières

1. [Importation des Librairies et Données](#1)
2. [Séparation de la Base](#2)
   - 2.1 [Création des jeux d'entraînement et de test](#2.1)
3. [Analyse de Départ](#3)
   - 3.1 [Aperçu des données](#3.1)
   - 3.2 [Répartition de la variable Cible](#3.2)
   - 3.3 [Descriptif des variables](#3.3)
   - 3.4 [Nombre de modalités par variable catégorielle](#3.4)
   - 3.5 [Types des variables](#3.5)
   - 3.6 [Conversion des dates](#3.6)
   - 3.7 [Données manquantes](#3.7)
4. [Analyse des Variables](#4)
   - 4.1 [Variables Binaires](#4.1)
   - 4.2 [Variables Quantitatives](#4.2)
   - 4.3 [Transformation des Variables de Date](#4.3)
5. [Traitement des Valeurs Manquantes](#5)
6. [Visualisation et Conclusion](#6)


## 1. 📥 Importation des Librairies et Données <a name="1"></a>

In [17]:
# Import des librairies

import pandas as pd

import numpy as np

import random

from datetime import datetime

import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline

# Import de la base de données

df = pd.read_csv('base.csv')

## 2. ✂️ Séparation de la Base <a name="2"></a>

### 2.1 Création des jeux d'entraînement et de test <a name="2.1"></a>

Nous appliquons la technique de hold-out pour diviser la base en deux parties : une base d'entraînement et une base de test. La base de test doit être un échantillon représentant 10% de la base. Après cela, nous sauvegardons les deux jeux de données dans des fichiers CSV pour une utilisation ultérieure.

In [12]:
# Base test

test = df.sample(frac=0.1, random_state=42)

# Export de la base de test

test.to_csv("test_raw_copy.csv", index=False)

# Base d'entraînement

train = df.drop(test.index)

# Export de la base d'entraînement

train.to_csv("train_raw.csv_copy", index=False)

## 3. 🔍 Analyse de Départ <a name="3"></a>

### 3.1 Aperçu des données <a name="3.1"></a>

Nous travaillerons uniquement avec la base train, la base test est laissée de côté.

Visualisons les premières lignes de notre dataframe afin d'avoir une vision d'ensemble de nos variables.

In [4]:
# Affichage des 10 premières lignes

train.head(10)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Vehicle_Date,Signature_Date
0,1,NaN,44,1,28,0,1,40454,26.0,217,1,1411164000,1.550099e+12
1,2,0.0,76,1,3,0,0,33536,26.0,183,0,1488841200,NaN
2,3,0.0,47,1,28,0,1,38294,26.0,27,1,1459202400,1.570486e+12
3,4,0.0,21,1,11,1,0,28619,152.0,203,0,1515884400,NaN
4,5,1.0,29,1,41,1,0,27496,152.0,39,0,1528754400,NaN
5,6,1.0,24,1,33,0,1,2630,160.0,176,0,1526594400,NaN
6,7,0.0,23,1,11,0,1,23367,152.0,249,0,1537999200,NaN
7,8,1.0,56,1,28,0,1,32031,26.0,72,1,1504562400,1.558735e+12
8,9,1.0,24,1,3,1,0,27619,152.0,28,0,1517958000,NaN
9,10,1.0,32,1,6,1,0,28771,152.0,80,0,1515970800,NaN


### 3.2 Répartition de la variable cible <a name="3.2"></a>

In [5]:
# Proportion de clients qui ont souscrit et qui n'ont pas souscrit 

train["Response"].value_counts()

Response
0    301071
1     41927
Name: count, dtype: int64

📌 La variable cible est déséquilibrée : peu de clients ont souscrit au produit.

### 3.3 Descriptif des variables <a name="3.3"></a>

- Qualitatives : Gender, Driving_License, Region_Code, Previously_Insured, Vehicle_Damage, Policy_Sales_Channel

- Quantitatives : Vintage, Age, Annual_Premium

- Dates (catégorielles ordinales) : Signature_Date, Vehicle_Date


Nombre de modalités par variable catégorielle <a name="3.4"></a>

In [9]:
# On affiche le nombre de catégories par variable

train[["Gender", "Driving_License", "Region_Code", "Previously_Insured",
       "Vehicle_Damage", "Policy_Sales_Channel"]].nunique()

Gender                    2
Driving_License           2
Region_Code              53
Previously_Insured        2
Vehicle_Damage            2
Policy_Sales_Channel    154
dtype: int64

On constate qu'il y a 4 variables binaires et 2 variables avec plus de 2 catégories : Region_Code et Policy_Sales_Channel. Ces informations nous serons utiles lorsque nous allons devoir analyser ces variables plus attentivement dans la suite du notebook.

Nous allons maintenant vérifier le type des variables présentes dans la base ainsi que le nombre de valeurs manquantes.

### 3.5 Types des variables <a name="3.5"></a>

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 342998 entries, 0 to 381108
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    342998 non-null  int64  
 1   Gender                325846 non-null  float64
 2   Age                   342998 non-null  int64  
 3   Driving_License       342998 non-null  int64  
 4   Region_Code           342998 non-null  int64  
 5   Previously_Insured    342998 non-null  int64  
 6   Vehicle_Damage        342998 non-null  int64  
 7   Annual_Premium        342998 non-null  int64  
 8   Policy_Sales_Channel  325764 non-null  float64
 9   Vintage               342998 non-null  int64  
 10  Response              342998 non-null  int64  
 11  Vehicle_Date          342998 non-null  int64  
 12  Signature_Date        41927 non-null   float64
dtypes: float64(3), int64(10)
memory usage: 36.6 MB


On constate qu'il n'y a que des variables numériques et qu'il n'y a donc pas de variables textuelles dans la base. 
A noter qu'en présence de variables textuelles, un réencodage est nécessaire car la plupart des algorithmes ne prennent pas en charge les variables non numériques.

Nous remarquons également que nos deux variables de date Vehicle_Date et Signature_Date ne sont pas de type date.


### 3.6 Conversion des dates <a name="3.6"></a>

In [13]:
# Timestamp exprimé en secondes

train["Vehicle_Date"] = pd.to_datetime(train["Vehicle_Date"], unit='s')

# Timestamp exprimé en millisecondes

train["Signature_Date"] = pd.to_datetime(train["Signature_Date"], unit='ms')

### 3.7 Données manquantes <a name="3.7"></a>

In [16]:
# Affichage des valeurs manquantes pour chaque variable 
train.isna().sum()/len(train)

id                      0.000000
Gender                  0.050006
Age                     0.000000
Driving_License         0.000000
Region_Code             0.000000
Previously_Insured      0.000000
Vehicle_Damage          0.000000
Annual_Premium          0.000000
Policy_Sales_Channel    0.050245
Vintage                 0.000000
Response                0.000000
Vehicle_Date            0.000000
Signature_Date          0.877763
dtype: float64

🔍 Trois variables comportent des valeurs manquantes : Gender, Policy_Sales_Channel, et Signature_Date.